In [1]:
import copy
import os
import cobra
from tests import TEST_DIR

aragem_model_path = os.path.join(TEST_DIR, 'data', 'AraGEM2010.xml')

non_diel_model = cobra.io.read_sbml_model(aragem_model_path)

In [2]:
non_diel_model_copy = copy.deepcopy(non_diel_model)

from diel_models.compartments_creator import CompartmentsCreator
from diel_models.storage_pool_creator import StoragePoolCreator
from diel_models.photon_reaction_restrictor import PhotonReactionRestrictor
from diel_models.biomass_adjuster import BiomassAdjuster

TEST = CompartmentsCreator(non_diel_model_copy)

In [3]:
TEST.day_attribution()

[<Metabolite S(43)45__40_3S_44_4R_41__45_cis_45_3_44_4_45_Dihydroxy_45_3_44_4_45_dihydrofluorene_c_Day at 0x290536ac0a0>,
 <Metabolite S(43)45_7_45_Isojasmonic_32_acid_32_CoA_c_Day at 0x290536caeb0>,
 <Metabolite S(45)45_Epiafzelechin_c_Day at 0x290534b99a0>,
 <Metabolite S(45)45_Epicatechin_c_Day at 0x290534b9970>,
 <Metabolite S(45)45_Epigallocatechin_c_Day at 0x290534b9a90>,
 <Metabolite S(45)45_Ureidoglycolate_c_Day at 0x290535bb520>,
 <Metabolite S(45)45_alpha_45_Pinene_c_Day at 0x29053438400>,
 <Metabolite S(45)45_beta_45_Pinene_c_Day at 0x29053438070>,
 <Metabolite S__40_11Z_44_14Z_41__45_Icosadienoyl_45_CoA_c_Day at 0x290536062b0>,
 <Metabolite S__40_11Z_44_14Z_44_17Z_41__45_Icosatrienoyl_45_CoA_c_Day at 0x290536065b0>,
 <Metabolite S__40_13Z_41__45_Docosenoic_32_acid_c_Day at 0x290535f9e20>,
 <Metabolite S__40_13Z_44_16Z_41__45_Docosadi_45_13_44_16_45_enoyl_45_CoA_c_Day at 0x29053614850>,
 <Metabolite S__40_15S_41__45_15_45_Hydroxy_45_5_44_8_44_11_45_cis_45_13_45_trans_45_eico

In [4]:
TEST.duplicate()

Name,MODEL1507180028
Memory address,29052302ca0
Number of metabolites,3466
Number of reactions,3202
Number of genes,1404
Number of groups,0
Objective expression,0.01*Ex16_Day - 0.01*Ex16_Day_reverse_efba9
Compartments,"Cytosol Day, Plastid Day, Mitochondrion Day, Peroxisome Day, Vacuole Day, Biomass Day, Accumulation Day, External Day, Cytosol Night, Biomass Night, Plastid Night, Accumulation Night, External Night, Mitochondrion Night, Vacuole Night, Peroxisome Night"


In [5]:
non_diel_model_copy

Name,MODEL1507180028
Memory address,29052302ca0
Number of metabolites,3466
Number of reactions,3202
Number of genes,1404
Number of groups,0
Objective expression,0.01*Ex16_Day - 0.01*Ex16_Day_reverse_efba9
Compartments,"Cytosol Day, Plastid Day, Mitochondrion Day, Peroxisome Day, Vacuole Day, Biomass Day, Accumulation Day, External Day, Cytosol Night, Biomass Night, Plastid Night, Accumulation Night, External Night, Mitochondrion Night, Vacuole Night, Peroxisome Night"


In [47]:
TEST2 = StoragePoolCreator(non_diel_model_copy, metabolites = ["S_Sucrose_c_Day", "S_Sulfate_c_Day", "S_Nitrate_c_Day", "S_L_45_Histidine_c_Day", "S_L_45_Isoleucine_c_Day", "S_L_45_Leucine_c_Day", "S_L_45_Lysine_c_Day", "S_L_45_Methionine_c_Day", "S_L_45_Phenylalanine_c_Day", "S_L_45_Threonine_c_Day", "S_L_45_Tryptophan_c_Day", "S_L_45_Valine_c_Day", "S_L_45_Arginine_c_Day", "S_L_45_Cysteine_c_Day", "S_L_45_Glutamine_c_Day", "S_L_45_Glutamate_c_Day", "S_Glycine_c_Day", "S_L_45_Proline_c_Day", "S_L_45_Tyrosine_c_Day", "S_L_45_Alanine_c_Day", "S_L_45_Asparagine_c_Day", "S_L_45_Serine_c_Day", "S_Orthophosphate_c_Day", "S_Starch_p_Day", "S_D_45_Fructose_c_Day", "S__40_S_41__45_Malate_c_Day", "S_Fumarate_c_Day", "S_Citrate_c_Day"])

In [48]:
TEST2.create_storage_pool_metabolites()
TEST2.create_storage_pool_first_reactions()
TEST2.create_storage_pool_second_reactions()

In [52]:
TEST3 = PhotonReactionRestrictor(non_diel_model_copy, "Ex16_Night")

In [53]:
TEST3.restrain()

In [57]:
for m in non_diel_model_copy.metabolites.query("chlorophyl"):
    if "Night" in m.compartment:
        print(m.reactions)

frozenset({<Reaction R03845_c_Night at 0x290532eafa0>})
frozenset({<Reaction R06267_c_Night at 0x29053d34fa0>, <Reaction R06286_c_Night at 0x29053d3abb0>})
frozenset({<Reaction R06286_c_Night at 0x29053d3abb0>})


In [56]:
for m in non_diel_model_copy.metabolites.query("carotene"):
    if "Night" in m.compartment:
        print(m.reactions)

frozenset({<Reaction R07856_c_Night at 0x29053f0ce20>, <Reaction R04803_c_Night at 0x29053176430>, <Reaction R04801_c_Night at 0x290531c0cd0>})
frozenset({<Reaction R06960_c_Night at 0x29053d80a90>, <Reaction R06961_c_Night at 0x29053d80ac0>})
frozenset({<Reaction R07856_c_Night at 0x29053f0ce20>})


In [58]:
TEST4 = BiomassAdjuster(non_diel_model_copy, "BIO_L_Day", "BIO_L_Night", ["R06286_c_Night", "R06267_c_Night", "R03845_c_Night", "R07856_c_Night", "R04803_c_Night", "R04801_c_Night", "R06960_c_Night", "R06961_c_Night", "R07856_c_Night"])

In [59]:
TEST4.reset_boundaries()

In [60]:
TEST4.total_biomass_reaction()

Reaction identifier,Biomass_Total
Name,Total Biomass Reaction
Memory address,0x2905338e640
Stoichiometry,0.04 S_4_45_Coumaryl_32_alcohol_biomass_Day + 0.04 S_4_45_Coumaryl_32_alcohol_biomass_Night + 0.0029 S_ATP_biomass_Day + 0.0029 S_ATP_biomass_Night + 30.0 S_ATP_c_Day + 30.0 S_ATP_c_Night + 0.0033... 0.04 4-Coumaryl alcohol_biomass Day + 0.04 4-Coumaryl alcohol_biomass Night + 0.0029 ATP_biomass Day + 0.0029 ATP_biomass Night + 30.0 ATP_c Day + 30.0 ATP_c Night + 0.0033 CTP_biomass Day + 0.0033...
GPR,
Lower bound,0
Upper bound,1000


In [66]:
cobra.io.write_sbml_model(non_diel_model_copy, os.path.join(TEST_DIR, "data", "Diel_Model_AraGEM.xml"))

In [64]:
FBA = non_diel_model_copy.optimize()

In [65]:
for reaction in non_diel_model_copy.reactions:
    print(reaction.id, FBA.fluxes[reaction.id])

Bc1_Day 1.4532122681386224
Bc2_Day 2.6322335422888257
Bc3_Day 8.36282531664679
Bc4_Day 7.266061340693112
Bc5_Day 7.732186030473426
Bc6_Day 10.693448765548357
Bc7_Day 2.9338436356760864
Bc8_Day 4.441894102612394
Bc9_Day 3.8386739158378704
Bc10_Day 5.758010873756805
Bc11_Day 2.3580425483004066
Bc12_Day 0.05483819879768386
Bc13_Day 0.6580583855722065
Bc14_Day 2.7419099398841937
Bc15_Day 3.8386739158378704
Bc16_Day 1.6451459639305162
Bc17_Day 9.596684789594676
Bc18_Day 0.0
Bc19_Day 0.27419099398841934
Bc20_Day 0.027419099398841938
Bc21_Day 0.0
Bc22_Day 2.1935279519073543
Bc23_Day 0.822572981965258
Bc24_Day 1.0967639759536776
Bc25_Day 10.693448765548357
Bc26_Day 1.0967639759536774
Bc27_Day 1.316116771144413
Bc28_Day 1.5354695663351483
Bc29_Day 1.4532122681386224
Bc30_Day 0.7128965843698905
Bc31_Day 0.07403156837687322
Bc32_Day 0.046612468978031285
Bc33_Day 0.05209628885779968
Bc34_Day 0.07677347831675742
Bc35_Day 0.0795153882566416
Bc36_Day 0.10693448765548354
Bc37_Day 0.09048302801617837
B

In [21]:
from tests import TEST_DIR

multi_tissue = os.path.join(TEST_DIR, 'data', 'multi-tissue_2018_working.xml')

In [22]:
model_multi_tissue = cobra.io.read_sbml_model(multi_tissue)

'CPD-1905_c_Leaf_Day' is not a valid SBML 'SId'.
'CPD-1909_c_Leaf_Day' is not a valid SBML 'SId'.
'CPD-1905_c_Leaf_Night' is not a valid SBML 'SId'.
'CPD-1909_c_Leaf_Night' is not a valid SBML 'SId'.
'CPD-1909_c_Root_Day' is not a valid SBML 'SId'.
'CPD-1905_c_Root_Day' is not a valid SBML 'SId'.
'CPD-1909_c_Root_Night' is not a valid SBML 'SId'.
'CPD-1905_c_Root_Night' is not a valid SBML 'SId'.
'--MENTHOL_c_Leaf_Day' is not a valid SBML 'SId'.
'--MENTHOL_c_Leaf_Night' is not a valid SBML 'SId'.
'--MENTHOL_c_Root_Day' is not a valid SBML 'SId'.
'--MENTHOL_c_Root_Night' is not a valid SBML 'SId'.
'4-ALPHA-METHYL-5-ALPHA-CHOLEST-7-EN-3-ON_c_Leaf_Day' is not a valid SBML 'SId'.
'CARBON-DIOXIDE_c_Leaf_Day' is not a valid SBML 'SId'.
'CPD-5846_c_Leaf_Day' is not a valid SBML 'SId'.
'CPD-5846_c_Leaf_Night' is not a valid SBML 'SId'.
'CARBON-DIOXIDE_c_Leaf_Night' is not a valid SBML 'SId'.
'4-ALPHA-METHYL-5-ALPHA-CHOLEST-7-EN-3-ON_c_Leaf_Night' is not a valid SBML 'SId'.
'4-ALPHA-METHYL-5-AL

In [26]:
model_multi_tissue.reactions.query("transport")

[<Reaction ALA_CP_Leaf_Day_rev_transport at 0x29067d4f8b0>,
 <Reaction ALA_CP_Leaf_Day_transport at 0x29067cc0580>,
 <Reaction ALA_CP_Leaf_Night_rev_transport at 0x29067cf70d0>,
 <Reaction ALA_CP_Leaf_Night_transport at 0x29067cf7040>,
 <Reaction ALA_CP_Root_Day_rev_transport at 0x29067d5afd0>,
 <Reaction ALA_CP_Root_Day_transport at 0x29067d5aeb0>,
 <Reaction ALA_CP_Root_Night_rev_transport at 0x29067d5a6a0>,
 <Reaction ALA_CP_Root_Night_transport at 0x29067d5af10>,
 <Reaction ARG_CP_Leaf_Day_rev_transport at 0x29067ddee20>,
 <Reaction ARG_CP_Leaf_Day_transport at 0x29067ddefa0>,
 <Reaction ARG_CP_Leaf_Night_rev_transport at 0x29067ddedc0>,
 <Reaction ARG_CP_Leaf_Night_transport at 0x29067ddeac0>,
 <Reaction ARG_CP_Root_Day_rev_transport at 0x29067ddea30>,
 <Reaction ARG_CP_Root_Day_transport at 0x29067ddef70>,
 <Reaction ARG_CP_Root_Night_rev_transport at 0x29067dea730>,
 <Reaction ARG_CP_Root_Night_transport at 0x29067dea850>,
 <Reaction ASN_CP_Leaf_Day_rev_transport at 0x29067d7bf1

In [63]:
model_multi_tissue.optimize()

,fluxes,reduced_costs
EX_NADP_c_Leaf_Day,0.0,0.0
EX_NADP_c_Leaf_Night,0.0,0.0
EX_NADP_c_Root_Day,0.0,0.0
EX_NADP_c_Root_Night,0.0,0.0
EX_NAD_c_Leaf_Day,0.0,0.0
...,...,...
sUREA_biomass_Root_Night,0.0,-0.0
sVAL_biomass_Leaf_Day,0.0,-0.0
sVAL_biomass_Leaf_Night,0.0,-0.0
sVAL_biomass_Root_Day,0.0,-0.0
